## Load up initial data

In [1]:
import pandas as pd
data_df = pd.read_csv('data/hourly_wages.csv')

In [2]:
data_df.head()

,wage_per_hour,union,education_yrs,experience_yrs,age,female,marr,south,manufacturing,construction
0,5.10,0,8,21,35,1,1,0,1,0
1,4.95,0,9,42,57,1,1,0,1,0
2,6.67,0,12,1,19,0,0,0,1,0
3,4.00,0,12,4,22,0,0,0,0,0
4,7.50,0,12,17,35,0,1,0,0,0


In [3]:
data_df.describe()

,wage_per_hour,union,education_yrs,experience_yrs,age,female,marr,south,manufacturing,construction
count,534.000000,534.000000,534.000000,534.000000,534.000000,534.000000,534.000000,534.000000,534.000000,534.000000
mean,9.024064,0.179775,13.018727,17.822097,36.833333,0.458801,0.655431,0.292135,0.185393,0.044944
std,5.139097,0.384360,2.615373,12.379710,11.726573,0.498767,0.475673,0.455170,0.388981,0.207375
min,1.000000,0.000000,2.000000,0.000000,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5.250000,0.000000,12.000000,8.000000,28.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,7.780000,0.000000,12.000000,15.000000,35.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,11.250000,0.000000,15.000000,26.000000,44.000000,1.000000,1.000000,1.000000,0.000000,0.000000
max,44.500000,1.000000,18.000000,55.000000,64.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [4]:
target = data_df.wage_per_hour.as_matrix()
predictors = data_df.drop(['wage_per_hour'], axis=1).as_matrix()

In [5]:
n_cols = predictors.shape[1]

## My First Model

In [6]:
from keras.models import Sequential
from keras.layers import Dense

my_model = Sequential()
my_model.add(Dense(100, activation='relu', input_shape=(n_cols,)))
my_model.add(Dense(100, activation='relu'))
my_model.add(Dense(1))

my_model.summary()

Using TensorFlow backend.


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_1 (Dense)                  (None, 100)           1000        dense_input_1[0][0]              
____________________________________________________________________________________________________
dense_2 (Dense)                  (None, 100)           10100       dense_1[0][0]                    
____________________________________________________________________________________________________
dense_3 (Dense)                  (None, 1)             101         dense_2[0][0]                    
Total params: 11,201
Trainable params: 11,201
Non-trainable params: 0
____________________________________________________________________________________________________


In [7]:
my_model.compile(optimizer = 'adam', loss='mean_squared_error')

In [8]:
my_model.fit(predictors, target)

Epoch 1/10
534/534 [==============================] - 0s - loss: 70.0865     
Epoch 2/10
534/534 [==============================] - 0s - loss: 26.9606     
Epoch 3/10
534/534 [==============================] - 0s - loss: 22.5278     
Epoch 4/10
534/534 [==============================] - 0s - loss: 21.6223     
Epoch 5/10
534/534 [==============================] - 0s - loss: 21.7286     
Epoch 6/10
534/534 [==============================] - 0s - loss: 22.1453     
Epoch 7/10
534/534 [==============================] - 0s - loss: 21.0051     
Epoch 8/10
534/534 [==============================] - 0s - loss: 21.1930     
Epoch 9/10
534/534 [==============================] - 0s - loss: 21.3519     
Epoch 10/10
534/534 [==============================] - 0s - loss: 21.6887     


## Validation and Early Stopping

In [9]:
def get_new_model():
    my_model = Sequential()
    my_model.add(Dense(100, activation='relu', input_shape=(n_cols,)))
    my_model.add(Dense(100, activation='relu'))
    my_model.add(Dense(1))
    my_model.compile(optimizer = 'adam', loss='mean_squared_error')
    return(my_model)
my_model = get_new_model()
my_model.fit(predictors, target, validation_split=0.3)

Train on 373 samples, validate on 161 samples
Epoch 1/10
373/373 [==============================] - 0s - loss: 49.8994 - val_loss: 34.6380
Epoch 2/10
373/373 [==============================] - 0s - loss: 28.0559 - val_loss: 30.1449
Epoch 3/10
373/373 [==============================] - 0s - loss: 22.5016 - val_loss: 28.7867
Epoch 4/10
373/373 [==============================] - 0s - loss: 20.4312 - val_loss: 24.9016
Epoch 5/10
373/373 [==============================] - 0s - loss: 19.7778 - val_loss: 25.6970
Epoch 6/10
373/373 [==============================] - 0s - loss: 19.5629 - val_loss: 25.0543
Epoch 7/10
373/373 [==============================] - 0s - loss: 19.8188 - val_loss: 25.2674
Epoch 8/10
373/373 [==============================] - 0s - loss: 19.4770 - val_loss: 25.2308
Epoch 9/10
373/373 [==============================] - 0s - loss: 19.4184 - val_loss: 25.6940
Epoch 10/10
373/373 [==============================] - 0s - loss: 19.3741 - val_loss: 25.4255


In [10]:
from keras.callbacks import EarlyStopping

early_stopping_monitor = EarlyStopping(patience=2)
my_model = get_new_model()
my_model.fit(predictors, target, validation_split=0.3, nb_epoch=20, callbacks=[early_stopping_monitor])

Train on 373 samples, validate on 161 samples
Epoch 1/20
373/373 [==============================] - 0s - loss: 57.9855 - val_loss: 37.6224
Epoch 2/20
373/373 [==============================] - 0s - loss: 27.6005 - val_loss: 35.2617
Epoch 3/20
373/373 [==============================] - 0s - loss: 22.1495 - val_loss: 26.1218
Epoch 4/20
373/373 [==============================] - 0s - loss: 20.6510 - val_loss: 25.7686
Epoch 5/20
373/373 [==============================] - 0s - loss: 20.2414 - val_loss: 25.6822
Epoch 6/20
373/373 [==============================] - 0s - loss: 19.9175 - val_loss: 25.4558
Epoch 7/20
373/373 [==============================] - 0s - loss: 19.8169 - val_loss: 25.9288
Epoch 8/20
373/373 [==============================] - 0s - loss: 19.6836 - val_loss: 25.4549
Epoch 9/20
373/373 [==============================] - 0s - loss: 19.4733 - val_loss: 26.3522
Epoch 10/20
373/373 [==============================] - 0s - loss: 19.9091 - val_loss: 27.3344
Epoch 11/20
373/373 [==

## Classification

In [11]:
titanic_data = pd.read_csv('data/titanic_all_numeric.csv')
titanic_data.head()

,survived,pclass,age,sibsp,parch,fare,male,age_was_missing,embarked_from_cherbourg,embarked_from_queenstown,embarked_from_southampton
0,0,3,22.0,1,0,7.2500,1,False,0,0,1
1,1,1,38.0,1,0,71.2833,0,False,1,0,0
2,1,3,26.0,0,0,7.9250,0,False,0,0,1
3,1,1,35.0,1,0,53.1000,0,False,0,0,1
4,0,3,35.0,0,0,8.0500,1,False,0,0,1


In [12]:
from keras.utils.np_utils import to_categorical
target = to_categorical(titanic_data.survived)
predictors = titanic_data.drop(['survived'], axis=1).as_matrix()
n_cols = predictors.shape[1]

def get_classification_model(n_cols):
    model = Sequential()
    model.add(Dense(100, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return(model)

my_model = get_classification_model(n_cols)
my_model.fit(predictors, target, validation_split=0.3, nb_epoch=20, callbacks=[early_stopping_monitor])

Train on 623 samples, validate on 268 samples
Epoch 1/20
623/623 [==============================] - 0s - loss: 0.7890 - acc: 0.6100 - val_loss: 0.7363 - val_acc: 0.6679
Epoch 2/20
623/623 [==============================] - 0s - loss: 0.6979 - acc: 0.6421 - val_loss: 0.5570 - val_acc: 0.7313
Epoch 3/20
623/623 [==============================] - 0s - loss: 0.6467 - acc: 0.6934 - val_loss: 0.5769 - val_acc: 0.6754
Epoch 4/20
623/623 [==============================] - 0s - loss: 0.6333 - acc: 0.6693 - val_loss: 0.5371 - val_acc: 0.7425
Epoch 5/20
623/623 [==============================] - 0s - loss: 0.5902 - acc: 0.6886 - val_loss: 0.4926 - val_acc: 0.7425
Epoch 6/20
623/623 [==============================] - 0s - loss: 0.5878 - acc: 0.6902 - val_loss: 0.5084 - val_acc: 0.7425
Epoch 7/20
623/623 [==============================] - 0s - loss: 0.5719 - acc: 0.7175 - val_loss: 0.4732 - val_acc: 0.7649
Epoch 8/20
623/623 [==============================] - 0s - loss: 0.5835 - acc: 0.7127 - val_l

## Convolutional Networks (Working with Images)

In [13]:
from keras.datasets import mnist

nb_classes = 10

# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# input image dimensions
img_rows, img_cols = 28, 28

# Using tf dim ordering
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = to_categorical(y_train, nb_classes)
Y_test = to_categorical(y_test, nb_classes)


X_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [14]:
from keras.layers import Dense, Flatten, Convolution2D, MaxPooling2D

# number of convolutional filters to use
nb_filters = 8
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (2, 2)

model = Sequential()
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape, activation='relu'))
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1], activation='relu'))
model.add(MaxPooling2D(pool_size=pool_size))

model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(nb_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

print(model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 27, 27, 8)     40          convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 26, 26, 8)     264         convolution2d_1[0][0]            
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 13, 13, 8)     0           convolution2d_2[0][0]            
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 1352)          0           maxpooling2d_1[0][0]             
___________________________________________________________________________________________

In [24]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
flatten_2 (Flatten)              (None, 8192)          0           flatten_input_1[0][0]            
____________________________________________________________________________________________________
dense_16 (Dense)                 (None, 256)           2097408     flatten_2[0][0]                  
____________________________________________________________________________________________________
dense_17 (Dense)                 (None, 1)             257         dense_16[0][0]                   
Total params: 2,097,665
Trainable params: 2,097,665
Non-trainable params: 0
____________________________________________________________________________________________________


In [15]:
model.fit(X_train, Y_train, nb_epoch=5, validation_data=(X_test, Y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 78s - loss: 0.2321 - acc: 0.9319 - val_loss: 0.0895 - val_acc: 0.9724
Epoch 2/5
60000/60000 [==============================] - 79s - loss: 0.0782 - acc: 0.9759 - val_loss: 0.0652 - val_acc: 0.9784
Epoch 3/5
60000/60000 [==============================] - 79s - loss: 0.0529 - acc: 0.9834 - val_loss: 0.0489 - val_acc: 0.9827
Epoch 4/5
60000/60000 [==============================] - 78s - loss: 0.0394 - acc: 0.9876 - val_loss: 0.0517 - val_acc: 0.9840
Epoch 5/5
60000/60000 [==============================] - 78s - loss: 0.0309 - acc: 0.9903 - val_loss: 0.0513 - val_acc: 0.9827


## Transfer Learning (using Pre-Trained Networks)
Loaded model from https://github.com/fchollet/deep-learning-models

In [16]:
from vgg16 import VGG16
from keras.preprocessing import image

model = VGG16(weights='imagenet', include_top=False)



K.image_dim_ordering: tf


In [17]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
block1_conv1 (Convolution2D)     (None, None, None, 64 1792        input_1[0][0]                    
____________________________________________________________________________________________________
block1_conv2 (Convolution2D)     (None, None, None, 64 36928       block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_pool (MaxPooling2D)       (None, None, None, 64 0           block1_conv2[0][0]               
___________________________________________________________________________________________

## Load very small set of cats and dogs data (from https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition)

In [19]:
from keras.preprocessing.image import ImageDataGenerator
train_data_dir = 'data/dogs_and_cats/train'
val_data_dir = 'data/dogs_and_cats/val'
n_cats_training = 40
n_dogs_training = 40
training_size = n_cats_training + n_dogs_training

n_cats_val = 10
n_dogs_val = 10
val_size = n_cats_val + n_dogs_val

datagen = ImageDataGenerator(rescale=1./255)


## Transform or "featurize" data with pretrained model

In [21]:
train_generator = datagen.flow_from_directory(
                            train_data_dir,
                            target_size=(150, 150),
                            class_mode=None,
                            shuffle=False) # keep data in order, since this is only a transform


val_generator = datagen.flow_from_directory(
                            val_data_dir,
                            target_size=(150, 150),
                            class_mode=None,
                            shuffle=False)

train_data = model.predict_generator(train_generator, training_size)
val_data = model.predict_generator(val_generator, val_size)


Found 80 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


In [22]:
import numpy as np
train_labels = np.array([0] * n_cats_training + [1] * n_dogs_training)
val_labels = np.array([0] * n_cats_val + [1] * n_dogs_val)


## Classification model baed on features of pre-trained model

In [23]:
model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_data, train_labels,
          nb_epoch=20,
          validation_data=(val_data, val_labels))


Train on 80 samples, validate on 20 samples
Epoch 1/20
80/80 [==============================] - 0s - loss: 1.5143 - acc: 0.5000 - val_loss: 2.1882 - val_acc: 0.5000
Epoch 2/20
80/80 [==============================] - 0s - loss: 1.5431 - acc: 0.5875 - val_loss: 0.6620 - val_acc: 0.6500
Epoch 3/20
80/80 [==============================] - 0s - loss: 0.4394 - acc: 0.7250 - val_loss: 0.9542 - val_acc: 0.5500
Epoch 4/20
80/80 [==============================] - 0s - loss: 0.1767 - acc: 0.9750 - val_loss: 0.3241 - val_acc: 0.8000
Epoch 5/20
80/80 [==============================] - 0s - loss: 0.2968 - acc: 0.8375 - val_loss: 0.3047 - val_acc: 0.8500
Epoch 6/20
80/80 [==============================] - 0s - loss: 0.1238 - acc: 0.9375 - val_loss: 0.5958 - val_acc: 0.8000
Epoch 7/20
80/80 [==============================] - 0s - loss: 0.0940 - acc: 0.9625 - val_loss: 1.0830 - val_acc: 0.6500
Epoch 8/20
80/80 [==============================] - 0s - loss: 0.0920 - acc: 0.9625 - val_loss: 0.6519 - val_